**Use**: Identifies the top-k hyper-parameters that yield the highest Auto-Labeling coverage (Coverage-Mean), while ensuring that the coverage remains within the error threshold defined by Auto-Labeling error (eps). 

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import itertools
sys.path.append('../')
sys.path.append('../../')

In [2]:
# Options
root_pfx = "../../../outputs/"
file_identifier = "mnist_lenet_calib_eval_hyp_2_std_xent_tbal_eval_hyp/tbal_train_time_search_mnist_lenet_calib_eval_hyp_2_std_xent_tbal_eval_hyp__01-25-2024__16-23-53"
eps_percent = 0.05 * 100


# Read xlsx file
df = pd.read_excel(f"{root_pfx}{file_identifier}.xlsx", sheet_name=0)
df = df.drop(columns=['Unnamed: 0'])

Filter records in dataframe

In [3]:
# Retain all records with Auto-Labeling-Err-Mean <= eps
df_1 = df.query("`Auto-Labeling-Err-Mean` <= @eps_percent")

# Sort by col: Coverage-Mean in descending order, and then by col: calib_conf in ascending order
df_1 = df_1.sort_values(["Coverage-Mean", "calib_conf"], ascending = [False, True])

# Change all calib_conf with NaN to "None"
df_1['calib_conf'] = df['calib_conf'].fillna("None")

# Sanity checks
assert df_1.query("`Auto-Labeling-Err-Mean` > @eps_percent").empty

In [4]:
df_1

,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,...,max_epochs,method,momentum,num_runs,optimizer,query_batch_frac,rank_target,rank_weight,seed_frac,weight_decay
13,None,crl,0.25,500,500,500,4.4312,56.44,6.0722,2.1461,...,50,tbal,0.9,5,sgd,0.04,softmax,0.8,0.4,0.010
22,None,crl,0.25,500,500,500,4.5818,56.40,6.0660,3.0198,...,50,tbal,0.9,5,sgd,0.04,softmax,0.9,0.4,0.010
25,None,crl,0.25,500,500,500,4.3675,55.76,4.4521,1.7586,...,50,tbal,0.9,5,sgd,0.04,softmax,0.9,0.4,0.010
3,None,crl,0.25,500,500,500,4.2372,55.64,4.1498,1.6806,...,50,tbal,0.9,5,sgd,0.04,softmax,0.7,0.4,0.001
12,None,crl,0.25,500,500,500,4.0315,54.56,3.8795,1.0393,...,50,tbal,0.9,5,sgd,0.04,softmax,0.8,0.4,0.001
15,None,crl,0.25,500,500,500,4.8115,54.08,8.3615,1.6278,...,50,tbal,0.9,5,sgd,0.04,softmax,0.8,0.4,0.001
21,None,crl,0.25,500,500,500,4.0654,54.04,5.0408,1.9636,...,50,tbal,0.9,5,sgd,0.04,softmax,0.9,0.4,0.001
7,None,crl,0.25,500,500,500,3.5651,53.68,3.1165,1.3038,...,50,tbal,0.9,5,sgd,0.04,softmax,0.7,0.4,0.010
4,None,crl,0.25,500,500,500,3.5188,53.28,5.7080,1.6277,...,50,tbal,0.9,5,sgd,0.04,softmax,0.7,0.4,0.010
16,None,crl,0.25,500,500,500,3.5960,50.96,3.5022,2.1684,...,50,tbal,0.9,5,sgd,0.04,softmax,0.8,0.4,0.010


In [5]:
# Set display option to show all columns
pd.set_option('display.max_columns', None)

train_time_methods = ['std_cross_entropy', 'crl', 'fmfp', 'squentropy']

# Find best training-time hyper-parameters
def print_top_k_train_time_methods(methods, methods_col_name, top_k):
    for method in methods:
        df_filtered = df_1[df_1[f'{methods_col_name}'] == method]
        df_filtered = df_filtered[df_filtered['calib_conf'] == 'None']
        if top_k == 1:
            # Retain ties for top-1
            max_value = df_filtered['Coverage-Mean'].max()
            df_2 = df_filtered[df_filtered['Coverage-Mean'] == max_value]
        else:
            df_2 = df_filtered.head(top_k)
        display(df_2)

In [6]:
# top k occurences for each method
top_k = 1 

print_top_k_train_time_methods(train_time_methods, "training_conf", top_k)

,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,batch_size,eps,learning_rate,max_epochs,method,momentum,num_runs,optimizer,query_batch_frac,rank_target,rank_weight,seed_frac,weight_decay


,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,batch_size,eps,learning_rate,max_epochs,method,momentum,num_runs,optimizer,query_batch_frac,rank_target,rank_weight,seed_frac,weight_decay
13,None,crl,0.25,500,500,500,4.4312,56.44,6.0722,2.1461,5.518,0.3404,32,0.05,0.01,50,tbal,0.9,5,sgd,0.04,softmax,0.8,0.4,0.01


,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,batch_size,eps,learning_rate,max_epochs,method,momentum,num_runs,optimizer,query_batch_frac,rank_target,rank_weight,seed_frac,weight_decay


,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,batch_size,eps,learning_rate,max_epochs,method,momentum,num_runs,optimizer,query_batch_frac,rank_target,rank_weight,seed_frac,weight_decay
